In [1]:
import random
import numpy as np
from itertools import groupby
import math

In [96]:
# Define the available orientations
orientations = [0, 45, -45, 90]

# Define the constraints
min_percentage = 0.1  # 10% of each orientation
max_group_size = 3    # maximum number of consecutive plies with the same orientation
max_imbalance = 2     # maximum difference between the number of plies with 0° and 90°

NUM_PLIES =16
MUTATION_RATE=.1
POP_SIZE = 200
NUM_ITERATIONS=1000
CROSS_RATE=.7

In [6]:
# Define the fitness function

def fitness_function(sequence):
    # Calculate the percentage of each orientation
    counts = {o: sequence.count(o) for o in orientations}
    percentages = {o: counts[o] / len(sequence) for o in orientations}
    
    # Calculate the number of groupings of plies with the same orientation
    groups = 0
    last_orientation = None
    group_size = 0
    for o in sequence:
        if o == last_orientation:
            group_size += 1
        else:
            groups += max(0, group_size - 1)
            last_orientation = o
            group_size = 1
    groups += max(0, group_size - 1)
    
    # Calculate the imbalance between 0° and 90°
    imbalance = abs(counts[0] - counts[90])
    
    # Rule 5: Penalize plies with fibers perpendicular to a free edge at the mid-plane
    edge_penalty = 0
    for i in range(len(sequence)):
        if sequence[i] in {0, 90}:
            if i == 0 or i == len(sequence) - 1:
                edge_penalty += 1
            elif sequence[i-1] != sequence[i] and sequence[i+1] != sequence[i]:
                edge_penalty += 1
    
    # Rule 6: Penalty for not alternating +θ° and -θ° plies
    alternating_penalty = 0
    last_angle = None
    for o in sequence:
        if o in {45, -45}:
            if last_angle is None:
                last_angle = o
            elif last_angle == -o:
                last_angle = o
            else:
                alternating_penalty += 1
        elif o == 0 or o == 90:
            last_angle = None
            
    # Rule 7: Penalty for grouping tape plies with the same orientation without a 45° ply in between
    grouping_penalty = 0
    last_orientation = None
    last_group_size = 0
    for o in sequence:
        if o == last_orientation:
            last_group_size += 1
        else:
            if last_group_size > 1 and last_orientation is not None:
                if sequence.index(o) - last_group_size < 0:
                    before = 45
                else:
                    before = sequence[sequence.index(o) - last_group_size - 1]
                if sequence.index(o) + 1 >= len(sequence):
                    after = 45
                else:
                    after = sequence[sequence.index(o) + 1]
                if abs(before - last_orientation) != 45 and abs(after - last_orientation) != 45:
                    grouping_penalty += 1
            last_orientation = o
            last_group_size = 1
    if last_group_size > 1 and last_orientation is not None:
        if abs(sequence[sequence.index(last_orientation) - last_group_size] - last_orientation) != 45:
            grouping_penalty += 1
            
    # Rule 8: Penalty for 0° plies too close to the surface
    surface_penalty = 0
    for i in range(len(sequence)):
        if sequence[i] == 0:
            if i < 3 or i > len(sequence) - 4:
                surface_penalty += 1
            elif 0 in sequence[i-3:i] or 0 in sequence[i+1:i+4]:
                surface_penalty += 1
    
    # Calculate the fitness value
    fitness = 1 / (1 + percentages[0] + percentages[45] + percentages[-45] + percentages[90] + 
                   imbalance + edge_penalty + grouping_penalty + surface_penalty)

    return fitness

In [17]:
def mutate(sequence):
    mutated_seq = sequence.copy()
    
    # Rule 1: Swap adjacent plies
    i = np.random.randint(len(sequence)-1)
    mutated_seq[i], mutated_seq[i+1] = mutated_seq[i+1], mutated_seq[i]
    
    # Rule 2: Flip orientation of a random ply
    i = np.random.randint(len(sequence))
    mutated_seq[i] = -mutated_seq[i]
    
    # Rule 3: Move a 90° ply to the end of the sequence
    if np.random.random() < 0.1 and 90 in sequence:
        mutated_seq.remove(90)
        mutated_seq.append(90)
    
    # Rule 4: Move a 0° ply to the middle of the sequence
    if np.random.random() < 0.1 and 0 in sequence:
        mutated_seq.remove(0)
        middle = len(mutated_seq) // 2
        mutated_seq.insert(middle, 0)
    
    # Rule 5: Swap ply at the mid-plane with ply 45° off-axis
    mid_plane = len(sequence) // 2
    if abs(mutated_seq[mid_plane]) == 90:
        if np.random.random() < 0.5 and 45 in sequence:
            for i in range(mid_plane+1, len(sequence)):
                if mutated_seq[i] == 45:
                    mutated_seq[mid_plane], mutated_seq[i] = mutated_seq[i], mutated_seq[mid_plane]
                    break
        else:
            for i in reversed(range(mid_plane)):
                if abs(mutated_seq[i]) == 45:
                    mutated_seq[mid_plane], mutated_seq[i] = mutated_seq[i], mutated_seq[mid_plane]
                    break
                    
    # Rule 6: Swap closest plies to mid-plane that are both θ° or -θ°
    if np.random.random() < 0.5:
        for i in range(mid_plane-1):
            if abs(mutated_seq[i]) == abs(mutated_seq[i+1]):
                mutated_seq[i], mutated_seq[i+1] = mutated_seq[i+1], mutated_seq[i]
                break
        for i in reversed(range(mid_plane+1, len(sequence)-1)):
            if abs(mutated_seq[i]) == abs(mutated_seq[i+1]):
                mutated_seq[i], mutated_seq[i+1] = mutated_seq[i+1], mutated_seq[i]
                break
    
    # Rule 7: Separate groups of tape plies of the same orientation with 45° plies
    for i in range(1, len(sequence)-1):
        if mutated_seq[i] == mutated_seq[i-1] and mutated_seq[i] == mutated_seq[i+1]:
            if abs(mutated_seq[i-1]) + abs(mutated_seq[i+1]) == 90:
                if mutated_seq[i] == 0:
                    mutated_seq[i], mutated_seq[i+1] = mutated_seq[i+1], mutated_seq[i]
                else:
                    mutated_seq[i], mutated_seq[i-1] = mutated_seq[i-1], mutated_seq[i]
                    
    # Rule 8: Move 0° plies at the surface closer to the middle of the sequence
    if 0 in mutated_seq:
        surface = mutated_seq.index(0)
        if surface < 3:
            for i in range(surface+1, len(sequence)):
                if mutated_seq[i] == 0 and i - surface >= 3:
                    # Swap the current 0° ply with the ply 3 positions closer to the middle
                    middle = len(mutated_seq) // 2
                    swap_index = max(surface+2, middle)
                    mutated_seq[swap_index], mutated_seq[i] = mutated_seq[i], mutated_seq[swap_index]
                    break
                elif mutated_seq[i] != 0:
                    break
    return mutated_seq


In [8]:
# Define the selection operator (tournament selection)
def selection(fitness_values, population_size):
    parents_indices = []
    for _ in range(population_size):
        indices = random.sample(range(len(fitness_values)), 2)
        if fitness_values[indices[0]] > fitness_values[indices[1]]:
            parents_indices.append(indices[0])
        else:
            parents_indices.append(indices[1])
    return parents_indices

In [9]:
# Define the crossover operator (single-point crossover)
def crossover(parent1, parent2, crossover_rate):
    if random.random() < crossover_rate:
        index = random.randint(1, len(parent1) - 1)
        child = parent1[:index] + parent2[index:]
    else:
        child = parent1
    return child

In [78]:
def mutate2(sequence):
    mutated_seq = sequence.copy()
    if np.random.random() < 0.1 and 90 in sequence:
        print(sequence)       
        mutated_seq.remove(90)
        mutated_seq.append(90)

In [81]:
# Define the genetic algorithm
def genetic_algorithm(population_size, sequence_length, crossover_rate, mutation_rate, max_generations):
    # Generate the initial population
    population = [[random.choice(orientations) for _ in range(sequence_length)] for _ in range(population_size)]
    
    # Evolve the population for a fixed number of generations
    for generation in range(max_generations):
        # Evaluate the fitness of each individual
        fitness_values = [fitness_function(individual) for individual in population]
        
        # Select the parents for reproduction
        parents_indices = selection(fitness_values, population_size)
        parents = [population[i] for i in parents_indices]
        
        # Generate the offspring by crossover and mutation
        offspring = []
        for i in range(population_size):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child = crossover(parent1, parent2, crossover_rate)
            child = mutate(child)
            offspring.append(child)
        
        # Replace the old population with the new offspring
        population = offspring
    
    # Find the best individual in the final population
    fitness_values = [fitness_function(individual) for individual in population]
    best_index = fitness_values.index(max(fitness_values))
    best_individual = population[best_index]
    
    return best_individual

In [90]:
# genetic_algorithm(POP_SIZE, NUM_PLIES, CROSS_RATE, MUTATION_RATE, NUM_ITERATIONS)

In [97]:
population = [[random.choice(orientations) for _ in range(NUM_PLIES)] for _ in range(POP_SIZE)]

In [98]:
for i in range(NUM_ITERATIONS):
# Calculate the fitness of each sequence
    fitnesses = [fitness_function(sequence) for sequence in population]
    # Select parents for crossover
    parent_indices = selection(fitnesses,POP_SIZE)

    # Generate the new population
    new_population = []
    for j in range(POP_SIZE // 2):
        parent1 = population[parent_indices[2*j]]
        parent2 = population[parent_indices[2*j+1]]
        if np.random.random() < CROSS_RATE:
            child1 = crossover(parent1, parent2, CROSS_RATE)
            child2 = crossover(parent2, parent1, CROSS_RATE)
        else:
            child1 = parent1
            child2 = parent2
        new_population.append(child1)
        new_population.append(child2)

    # Apply mutations to the new population
    for j in range(POP_SIZE):
        if np.random.random() < MUTATION_RATE:
            new_population[j] = mutate(new_population[j])

    # Replace the old population with the new population
    population = new_population

# Print the best sequence found so far
best_sequence = max(population, key=fitness_function)
print(f"Iteration {i+1}: Best fitness = {fitness_function(best_sequence)}, Best sequence = {best_sequence}")

Iteration 1000: Best fitness = 0.64, Best sequence = [45, -90, -45, -90, 45, -90, 0, -90, 45, 90, 0, -90, 45, -90, 90, -90]
